In [1]:
import requests
from bs4 import BeautifulSoup
from time import sleep
import csv
import random
from tqdm.notebook import tqdm as tqdm 
import numpy as np
import pandas as pd 
import json
import datetime
from scipy import stats
import time
from datetime import timedelta
import re
import string

In [2]:
courses = pd.read_csv('courses.csv')

In [3]:
courses.head()

,certificate_availability,cost,course_link,course_name,duration,language,overview,reviews,site,stars,sylabus
0,Certificate Available,Free Online Course,https://www.classcentral.com/course/independen...,Elements of AI,"6 weeks long, 5-10 hours a week",English,Overview The goal of this course is to demyst...,642,classcentral,4.8,Syllabus Part 1 What is AI? Part 2 Solving pr...
1,Paid Certificate Available,Free Online Course (Audit),https://www.classcentral.com/course/machine-le...,Machine Learning,"11 weeks long, 60 hours worth of material",English,Overview Class Central Tips Learn ...,375,classcentral,4.8,Syllabus IntroductionWelcome to Machine Learn...
2,$149.00 Certificate Available,Free Online Course (Audit),https://www.classcentral.com/course/edx-unlock...,Unlocking Information Security I: From Cryptog...,"5 weeks long, 4-6 hours a week",English,Overview Information Security is everywhere:...,237,classcentral,4.8,NaN
3,Paid Certificate Available,Free Online Course,https://www.classcentral.com/course/swayam-cyb...,Cyber Security,15 weeks long,English,Overview 1. Why this Course on Cybe...,236,classcentral,4.7,Syllabus COURSE LAYOUT Week - 11.Introduction...
4,Paid Certificate Available,Free Online Course (Audit),https://www.classcentral.com/course/informatio...,"Information Systems Auditing, Controls and Ass...","4 weeks long, 8 hours worth of material",English,Overview Class Central Tips Learn ...,208,classcentral,4.7,Syllabus Introduction to Information Systems ...


In [4]:
courses.shape

(43532, 11)

In [5]:
courses.sample(20)

,certificate_availability,cost,course_link,course_name,duration,language,overview,reviews,site,stars,sylabus
21673,Certificate Available,Free Trial Available,https://www.classcentral.com/course/linkedin-l...,Mograph Techniques: Creating a Product Endpage,1-2 hours worth of material,English,Overview Create a product endpage f...,NaN,classcentral,NaN,Syllabus IntroductionWelcomeUsing t...
40259,$99.00 Certificate Available,Free Online Course (Audit),https://www.classcentral.com/course/edx-landsc...,Landscape Governance: Collaborating Across Sec...,"4 weeks long, 1-3 hours a week",English,Overview Increasing global complexities and i...,NaN,classcentral,NaN,Syllabus 1: Introduction to Landscape Governa...
10341,Certificate Available,Paid Course,https://www.classcentral.com/course/udemy-micr...,Microsoft Excel: Excel - Beginner to Advanced ...,4-5 hours worth of material,English,Overview Became in a master of Exce...,NaN,classcentral,NaN,NaN
30724,Certificate Available,Free Trial Available,https://www.classcentral.com/course/pluralsigh...,Installing and Configuring Windows 10,3-4 hours worth of material,English,Overview Windows 10 is the best cli...,NaN,classcentral,NaN,NaN
2818,Certificate Available,Paid Course,https://www.classcentral.com/course/udemy-mach...,【徹底的に解説！】人工知能・機械学習エンジニア養成講座（初級編～統計学から数字認識まで～）,16 hours worth of material,Japanese,Overview 単回帰分析・ロジスティック回帰分析・ニューラルネット...,NaN,classcentral,NaN,NaN
33139,Certificate Available,Paid Course,https://www.classcentral.com/course/robotics-1...,Robotics,"30 weeks long, 5 hours a week",English,Overview Class Central Tips Learn ...,1,classcentral,1.0,Syllabus Course 1: Robotics: Aerial...
18119,Certificate Available,Free Trial Available,https://www.classcentral.com/course/linkedin-l...,Job Interview Strategies for UX Designers,Less than 1 hour of material,English,Overview Learn how to approach your...,NaN,classcentral,NaN,Syllabus IntroductionUX design inte...
35712,Paid Certificate Available,Free Online Course,https://www.classcentral.com/course/udemy-flat...,Flat Stomach and Core Strength for Women,3-4 hours worth of material,English,Overview 8 simple workouts from beg...,NaN,classcentral,NaN,NaN
28096,Certificate Available,Paid Course,https://www.classcentral.com/course/udemy-pyth...,Python Hero: Full Course with Projects ||Hindi...,9 hours worth of material,Hindi,Overview Most Comprehensive Python ...,NaN,classcentral,NaN,NaN
8406,Certificate Available,Paid Course,https://www.classcentral.com/course/udemy-how-...,EXCEL at Work - Complete MS Excel Mastery Begi...,7 hours worth of material,English,Overview Boost your Excel skills - ...,NaN,classcentral,NaN,NaN


In [9]:
response = requests.get('https://www.classcentral.com/subjects', timeout= 15)
sleep(random.uniform(1,2))
if response.status_code == 200:
    soup = BeautifulSoup(response.content, 'html.parser')

In [17]:
classes = soup.find_all('a', class_= 'border-box align-middle color-charcoal hover-no-underline')

In [54]:
hrefs = []
page_numbs = []
for cls in classes:
    hrefs.append('https://www.classcentral.com'+ cls.get('href'))
    page_numbs.append(round(int(cls.find('span', class_= 'text-1 icon-left-small icon-course-charcoal').text.strip().split()[0])/50))

In [41]:
hrefs

['https://www.classcentral.com/subject/cs',
 'https://www.classcentral.com/subject/business',
 'https://www.classcentral.com/subject/humanities',
 'https://www.classcentral.com/subject/data-science',
 'https://www.classcentral.com/subject/personal-development',
 'https://www.classcentral.com/subject/art-and-design',
 'https://www.classcentral.com/subject/programming-and-software-development',
 'https://www.classcentral.com/subject/engineering',
 'https://www.classcentral.com/subject/health',
 'https://www.classcentral.com/subject/maths',
 'https://www.classcentral.com/subject/science',
 'https://www.classcentral.com/subject/social-sciences',
 'https://www.classcentral.com/subject/education']

In [55]:
page_numbs

[98, 173, 52, 26, 32, 98, 175, 44, 36, 16, 41, 48, 35]

In [71]:
page_links= []
for href, page_numb in zip(hrefs,page_numbs):
    for i in range(page_numb):
        page_links.append(href +'?page='+ f'{i+1}')

In [108]:
missing = np.nan
df = pd.DataFrame()
for page_link in tqdm(page_links):
    response = requests.get(page_link, timeout= 15)
    sleep(random.uniform(1,2))
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        tags = soup.find_all('td', class_ = 'width-12-16 large-up-width-8-16 xxlarge-up-width-9-16 relative')
        final_tags = []
        for tag in tags:
            if not tag.get('data-track-impression'):
                final_tags.append(tag)
        for final_tag in final_tags:
            site = 'classcentral'
            try:
               course_name = final_tag.find('span', class_='text-1 weight-semi line-tight').text.strip()
            except:
                course_name = missing
            try:
                course_link = ('https://www.classcentral.com'+final_tag.find('a', class_='color-charcoal block line-tight course-name').get('href'))
            except:
                course_link = missing
            temp_dic1 = {'site' : site,
                  'course_name' : course_name,
                  'course_link' : course_link,}
            df =(df.append(temp_dic1, ignore_index=True))
    else:
        print('Response Status in page links not 200')

In [110]:
courses = pd.DataFrame()

In [111]:
for index, row in tqdm(df.iterrows()):
    course_link = row['course_link']
    try:
        response = requests.get(course_link, timeout= 15)
        sleep(random.uniform(2,3))
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            info = soup.find('div', class_= 'bg-white small-down-padding-medium padding-large border-all border-gray-light radius-small margin-bottom-medium')
            lis = info.find_all('li', class_ = 'nowrap padding-horz-small padding-vert-xsmall border-bottom border-gray-light row vert-align-top horz-align-left')
            section = soup.find('section', class_= 'bg-white border-all border-gray-light radius-small padding-large margin-bottom-medium')
            ps = section.find('p', class_ = 'text-1 medium-down-margin-top-xsmall large-up-margin-left-xsmall inline-block')
            overviewblock = soup.find('div', class_='bg-white border-all border-gray-light radius-small small-down-padding-medium padding-large margin-bottom-medium relative')
            try:
                for li in lis:
                    if 'language' in li.find('i').get('class')[0]:
                        language = li.find('a').text.strip()
            except:
                language = missing
            try:
                for li in lis:
                    if 'dollar' in li.find('i').get('class')[0]:
                        cost = li.find('span').text.strip()
            except:
                cost = missing
            try:
                for li in lis:
                    if 'credential' in li.find('i').get('class')[0]:
                        certificate_availability = li.find('span').text.strip()
            except:
                certificate_availability = missing
            try:
                for li in lis:
                    if 'clock' in li.find('i').get('class')[0]:
                        duration = li.find('span').text.strip()
            except:
                duration = missing
            try:
               stars = ps.find('strong', class_='weight-bold').text.strip()
            except:
                stars = missing
            try:
                reviews = ps.find_all('strong')[1].text.strip()
            except:
                reviews = missing
            try:
                overview = overviewblock.text.strip().replace('\n', ' ').replace('\xa02 ', ' ').replace('/', ' ').replace('\xa0', ' ')
            except:
                overview = missing
            try:
                sylabus = soup.find('section', class_= 'border-bottom border-gray-light padding-vert-large').text.strip().replace('\n', ' ')
            except:
                sylabus = missing
            temp_dic2  = {
                          'site' : row['site'],
                          'course_name' : row['course_name'],
                          'course_link' : row['course_link'],
                          'language' : language,
                          'cost' : cost,
                          'certificate_availability' : certificate_availability,
                          'duration' : duration,
                          'stars' : stars,
                          'reviews' : reviews,
                          'overview' : overview,
                          'sylabus' : sylabus}
            courses = courses.append(temp_dic2, ignore_index=True)
        else:
            print('Response Status in page links not 200')
    except:
        print('Connection error')

Connection error
Connection error
Connection error
Connection error
Connection error
Connection error
Connection error
Connection error
Connection error
Connection error
Connection error
Connection error
Connection error
Connection error
Connection error
Connection error
Connection error
Connection error
Connection error
Connection error
Connection error
Connection error
Connection error
Connection error
Connection error
Connection error
Connection error
Connection error
Connection error
Connection error
Connection error
Connection error
Connection error
Connection error
Connection error
Connection error
Connection error
Connection error
Connection error
Connection error
Connection error
Connection error
Connection error
Connection error
Connection error
Connection error
Connection error
Connection error
Connection error
Connection error
Connection error
Connection error
Connection error
Response Status in page links not 200
Connection error
Connection error
Connection error
Connection

In [112]:
courses.to_csv('courses.csv', index=False)

In [115]:
courses.tail()

,certificate_availability,cost,course_link,course_name,duration,language,overview,reviews,site,stars,sylabus
43527,Certificate Available,Free Online Course,https://www.classcentral.com/course/independen...,Integrated STEM Teaching for Secondary Schools,20 hours worth of material,English,Overview This MOOC aims to provide secondary ...,NaN,classcentral,NaN,Syllabus Module 1: Introduction to integra...
43528,Certificate Available,Free Online Course,https://www.classcentral.com/course/france-uni...,Se former pour enseigner dans le supérieur,2 hours a week,French,Overview À propos du cours Ce MOOC vise à so...,NaN,classcentral,NaN,Syllabus Plan du cours Thème 1 : Rendre les é...
43529,Certificate Available,Free Online Course,https://www.classcentral.com/course/france-uni...,Enseigner dans le réseau des établissements fr...,9 weeks long,French,Overview Évaluation Vous pourrez obtenir une ...,NaN,classcentral,NaN,Syllabus Plan du cours Partie 1 : Un contexte...
43530,Certificate Available,Free Online Course,https://www.classcentral.com/course/france-uni...,Le métier d'étudiant : pour une adaptation réu...,"29 weeks long, 1 hour a week",French,Overview À propos du cours Quand s’inscrire ?...,NaN,classcentral,NaN,Syllabus Plan du cours Au programme : des vid...
43531,Certificate Available,Free Online Course,https://www.classcentral.com/course/france-uni...,Étudiants dyslexiques dans mon amphi : compren...,"4 weeks long, 1 hour a week",French,Overview À propos du cours La dyslexie affect...,NaN,classcentral,NaN,Syllabus Plan du cours Le MOOC est organisé s...


In [75]:
courses['course_link'][0]

'https://www.classcentral.com/course/independent-elements-of-ai-12469'

In [74]:
df['course_link'][1000]

NameError: name 'df' is not defined